In [1]:
# import concurrent.futures
# import librosa
# import pandas as pd
import numpy as np
import cv2
# import matplotlib.pyplot as plt
# # import tensorflow as tf
# # import pickle
# # import json
# import librosa.display
# # from scipy import signal
from statistics import mode
# from scipy.io import wavfile
import os
# import skimage.io
from tqdm import tqdm
# from tensorflow.keras.applications.densenet import DenseNet121, DenseNet169
# from tensorflow.keras.applications.vgg16 import VGG16
# from tensorflow.keras.applications.resnet import ResNet101 
# from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import pickle
import random
from random import shuffle
import pandas as pd
import glob

from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import manhattan_distances, cosine_distances
from sklearn.metrics import silhouette_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.metrics import davies_bouldin_score
import pickle

In [2]:
with open('../pickle_files/cough_sound/cs_vgg16.pickle','rb') as handle:
   dataset = pickle.load(handle)

In [3]:
dataset=dataset[:1312]

In [4]:
random.seed(42)
shuffle(dataset)

In [5]:
def correct_mispredictions(query, fea_label,train_label, train_id, ind_data, decision,data_frame_1, count):
    if mode(decision) != query["label"]:
        count +=1 
        data_frame_1["Image name"].append(query["filepath"].split("/")[-1])
        data_frame_1["Mistake ID"].append(query['id'])
        data_frame_1["Original label"].append(query['label'])
        data_frame_1["Predicted label"].append(mode(decision))
        data_frame_1["Mistake index"].append(ind_data)
        fea_label[query['label']].append(query["feature"])
        train_label[query['label']].append(query["label"])
        train_id[query['label']].append(query['id'])
    else:
        fea_label[query['label']].append(query["feature"])
        train_label[query['label']].append(query["label"])
        train_id[query['label']].append(query['id'])
    return count,data_frame_1,fea_label,train_label,train_id

In [7]:
def distance1(query, fea_label, select_distance, id_pred, label_pred, n_neighbours,count,train_label, train_id, ind_data, data_frame_1,supervised_data): # Query is the raw dictionary (from pickle file) // fea_label is dictionary of {0: [], 1:[]} (distance) // select distance is int
  exp_query = np.expand_dims(query['feature'], axis=0)
  pos_tup, neg_tup = [], []
  
  if select_distance==1: # Euclidean distance
    
    neg_dist = np.linalg.norm(query['feature']-fea_label[0], axis=1)  # Calculating the Euclidean distance using numpy (axis=1) to calculate all at ones   
    pos_dist = np.linalg.norm(query['feature']-fea_label[1],axis=1)
  
  elif select_distance==2: # Manhattan distance
     neg_dist = np.squeeze(manhattan_distances(fea_label[0],exp_query))  # convert (1,n) to (,n)
     pos_dist=np.squeeze(manhattan_distances(fea_label[1],exp_query))

  elif select_distance==3: # Cosine distance
    neg_dist = np.squeeze(cosine_distances(exp_query,fea_label[0]))  # convert (1,n) to (,n)
    pos_dist=np.squeeze(cosine_distances(exp_query,fea_label[1]))

  
  for dist_single in pos_dist:
    pos_tup.append((dist_single,1))

  for dist_single in neg_dist:
    neg_tup.append((dist_single,0))

  pos_tup.extend(neg_tup)
  tup_dist = sorted(pos_tup)[:n_neighbours]
  
  
  decision = [y for (x,y) in tup_dist]
  if supervised_data:
    count,data_frame_1,fea_label,train_label,train_id=correct_mispredictions(query, fea_label,train_label,train_id, ind_data, decision,data_frame_1, count)
  
  else:
    if decision.count(0) > decision.count(1):
      fea_label[0].append(query["feature"])
      id_pred[0].append(query["id"])
      label_pred[0].append((query['id'],decision.count(1)/n_neighbours))

    else:
      fea_label[1].append(query["feature"])
      id_pred[1].append(query["id"])
      label_pred[1].append((query['id'],decision.count(1)/n_neighbours))
  
  return fea_label, id_pred, label_pred, data_frame_1, count, train_label, train_id

In [8]:
def classification_metrics(label_gt,id_pred):
  TP,FP,FN,TN = 0,0,0,0

  for tp in id_pred[1]:   # TP --> When correctly classified covid
    if tp in label_gt[1]:
      TP +=1

  for tn in id_pred[0]:  # TN --> When correctly classified healthy (non-covid)
    if tn in label_gt[0]:
      TN +=1

  for fp in id_pred[1]: # FP --> When incorrectly classified healthy (Classified healthy as covid)
    if fp in label_gt[0]:
      FP +=1

  for fn in id_pred[0]: # FN --> When missed covid classification (Covid cases missed)
    if fn in label_gt[1]:
      FN +=1

  accuracy= (TP+TN)/(TP+TN+FP+FN+1e-5)
  specificity = TN/(TN+FP+1e-5)
  sensitivity = (TP)/(TP+FN+1e-5)
  # f1_score = (2*precision*recall)/(precision + recall)
  
  print("TP: ", TP)
  print("FP: ", FP)
  print("FN: ", FN)
  print("TN: ", TN)

  return accuracy, specificity, sensitivity,TP,TN,FP,FN

def roc_auc_curve(label_gt,label_pred):
  gt_labels= sorted(label_gt[0]+ label_gt[1])  # Contains (id,labels) tuple of binary class 
  pred_labels = sorted(label_pred[0]+label_pred[1]) # Contains (id,labels) tuple of binary class --> sorted to match each element in gt_labels and pred_labels
  y_test = [y for (x,y) in gt_labels]   # Get only the labels
  y_scores = [y for (x,y) in pred_labels]
  fpr, tpr, threshold = roc_curve(y_test, y_scores)
  roc_auc = auc(fpr, tpr)
  return roc_auc

def cluster_metrics(fea_label,train_label,id_pred):
  print("Calculating Dunn's index...")
  intra_dist1 = euclidean_distances(fea_label[0]).max()
  intra_dist2 = euclidean_distances(fea_label[1]).max()
  inter_dist = euclidean_distances(fea_label[0],fea_label[1]).min()

  if intra_dist1>intra_dist2:
    max_intra_dist= intra_dist1  
  else:
    max_intra_dist = intra_dist2 

  Dunn_index = inter_dist/max_intra_dist

  print("Calculating Davies Bouldin index...")

  # Davies Bouldin and Silhouette score from sklearn library.
  class_0 =np.concatenate((np.zeros(shape=(len(train_label[0])),dtype=int),np.zeros(shape=(len(id_pred[0])),dtype=int)))
  class_1 = np.concatenate((np.ones(shape=(len(train_label[1])),dtype=int),np.ones(shape=(len(id_pred[1])),dtype=int)))
  class_all = np.concatenate((class_0,class_1))
  feature_all = np.concatenate((fea_label[0],fea_label[1]))

  davies_bouldin_index = davies_bouldin_score(feature_all,class_all)
  silhouette_index = silhouette_score(feature_all,class_all)

  print("davies: ", davies_bouldin_index)
  print("silhouette_sklearn: ", silhouette_index)
  
  return Dunn_index,davies_bouldin_index, silhouette_index

In [9]:
# labeled_size = [240,640,840,1250]
labeled_size = [240,1250]

In [28]:
# def data_loader(dataset,n): # Function to return three sets of labeled dataset for experiment
#   labeled_data, unlabeled_data = [], [] 

#   l_data = dataset[:n]    # First dataset // labeled
#   ul_data = dataset[n:]   # First dataset // unlabeled
#   labeled_data.append(l_data)
#   unlabeled_data.append(ul_data)

#   l_data = dataset[901:901+n]    # second dataset // labeled
#   ul_data = dataset[:901]+dataset[901+n:]
#   labeled_data.append(l_data)
#   unlabeled_data.append(ul_data)

#   l_data = dataset[1802:1802+n]     # Third dataset // labeled
#   ul_data = dataset[:1802]+dataset[1802+n:]
#   labeled_data.append(l_data)
#   unlabeled_data.append(ul_data)
#   return labeled_data, unlabeled_data

In [10]:
# def data_loader(dataset,n): # Function to return three sets of labeled dataset for experiment
#   labeled_data, unlabeled_data = [], [] 

#   l_data = dataset[:n]    # First dataset // labeled
#   ul_data = dataset[n:]   # First dataset // unlabeled
#   labeled_data.append(l_data)
#   unlabeled_data.append(ul_data)

#   l_data = dataset[437:437+n]    # second dataset // labeled
#   ul_data = dataset[:437]+dataset[437+n:]
#   labeled_data.append(l_data)
#   unlabeled_data.append(ul_data)

#   l_data = dataset[874:874+n]     # Third dataset // labeled
#   ul_data = dataset[:874]+dataset[874+n:]
#   labeled_data.append(l_data)
#   unlabeled_data.append(ul_data)
#   return labeled_data, unlabeled_data

In [10]:
def data_loader(dataset,n): # Function to return three sets of labeled dataset for experiment
  labeled_data, unlabeled_data = [], [] 

  l_data = dataset[:n]    # First dataset // labeled
  ul_data = dataset[n:]   # First dataset // unlabeled
  labeled_data.append(l_data)
  unlabeled_data.append(ul_data)

  l_data = dataset[200:200+n]    # second dataset // labeled
  ul_data = dataset[:200]+dataset[200+n:]
  labeled_data.append(l_data)
  unlabeled_data.append(ul_data)

  l_data = dataset[300:300+n]     # Third dataset // labeled
  ul_data = dataset[:300]+dataset[300+n:]
  labeled_data.append(l_data)
  unlabeled_data.append(ul_data)
  return labeled_data, unlabeled_data

In [24]:
def data_separation(dataset,label):
    add_data, tup_id_label= [], []
    i=0
    while len(add_data)!=20:
        # print(i)
        if dataset[i]["label"]==label:
            add_data.append(dataset[i]['feature'])
            tup_id_label.append((dataset[i]["id"],dataset[i]['label']))
            del dataset[i]
        i+=1
    return add_data, tup_id_label

In [25]:
n_neighbors=[5,9,15,19,25,31,39]
data_frame = {"Labeled data": [],
              "Dataset": [],
              "Accuracy": [],
              "Specificity": [],
              "Sensitivity": [],
              "AUC":[],
              # "Dunn index": [],
              # "Davies Bouldin": [],
              # "Silhouette index":[],
              "K": [],
              "TP":[],
              "TN":[],
              "FP":[],
              "FN":[],
              "pos_labeled_img":[],
              "neg_labeled_img":[],
              "corrected_count":[]
    
}

for size in labeled_size:
  labeled_data, unlabeled_data = data_loader(dataset, size)
  print(f"labeled data length {len(labeled_data)}")
  print(f"Unlabeled data length {len(unlabeled_data)}")
  select=0         # To select the dataset out of three sets ==> three sets: [d11, d12, d13] ==> eg: [200,200,200]
  
  while(select < 1):
    data_frame_1 = {  "Image name": [],
                  "Mistake index": [],
                  "Mistake ID": [],
                  "Original label": [],
                  "Predicted label": []
                  
}
    for neighbor in n_neighbors:
      labeled_data, unlabeled_data = data_loader(dataset, size)
      pos_img,neg_img=0,0
      # mis_predict_id = {0: [],    
      # 1 :[]}

      label_gt = {0: [],    
      1 :[]}    
                          # Collect the ground truth (label) of all the predicting images
      train_label = {0: [],    
      1 :[]}    

      label_pred = {0: [],
      1 :[]}               # Collect the predicted label for all the images

      id_gt = {0: [], 
          1: [] }         # Collect the ground truth (id) of all the predicting images

      id_pred = {0: [],
            1: []}        # Collect the predicted id for all the images 

      fea_label = {0: [],
            1: []}

      train_id ={0: [],
          1:[]}
      
      fpositive, ptup_id_label = data_separation(labeled_data[select],1)
      fnegative, ntup_id_label = data_separation(labeled_data[select],0)
      # training_data, supervised_data = labeled_data[select][:20], labeled_data[select][20:]
      
      # for data in labeled_data[select]:
      fea_label={ 0: fnegative,
                    1: fpositive
        }
      
      train_label = {0: ntup_id_label, 
                    1: ptup_id_label}
      
      # for data in training_data:

      #   if data["label"] == 1:
      #     fea_label[1].append(data['feature'])
      #     train_id[1].append(data['id'])
      #     train_label[1].append((data['id'],data['label']))
      #     pos_img +=1

      #   else:
      #     fea_label[0].append(data['feature'])
      #     train_id[0].append(data['id'])
      #     train_label[0].append((data['id'],data['label']))
      #     neg_img +=1
      
      print(f"Feature length neg: {len(fea_label[0])}")
      print(f"Feature length pos: {len(fea_label[1])} ")  

      # supervised_data= True
      count,ind_data=0,40
      for data in labeled_data[select]:
        fea_label, id_pred, label_pred, data_frame_1, count, train_label, train_id=distance1(data,fea_label,1,id_pred,label_pred,neighbor, count, train_label, train_id, ind_data, data_frame_1 ,supervised_data=True)
        
        ind_data +=1
      data_f_1 = pd.DataFrame.from_dict(data_frame_1)
      # data_f_1.to_csv(f"../cough_sound_csv/densenet169_cosine_mistake_{size}_{select}.csv",index=False)
        
      # # supervised_data = False
      for data in tqdm(unlabeled_data[select]):
        if data["label"]==1:
          id_gt[1].append(data['id'])
          label_gt[1].append((data['id'],data['label']))
        
        else:
          id_gt[0].append(data['id'])
          label_gt[0].append((data['id'],data['label']))

        fea_label,id_pred,label_pred,_,_,_,_ = distance1(data,fea_label,1,id_pred,label_pred,neighbor,count=None,train_label=None, train_id=None, ind_data=None, data_frame_1=None, supervised_data=False)  
      accuracy, specificity, sensitivity,TP,TN,FP,FN= classification_metrics(id_gt,id_pred)
      # dunn_index, davies_bouldin_index, silhouette_index = cluster_metrics(fea_label,train_label,id_pred)
      cl_auc = roc_auc_curve(label_gt,label_pred)
      data_frame["Labeled data"].append(size)
      data_frame["Dataset"].append(f"d_{select}")
      data_frame["Accuracy"].append(accuracy)
      data_frame["Specificity"].append(specificity)
      data_frame["Sensitivity"].append(sensitivity)
      data_frame["AUC"].append(cl_auc)
      # data_frame["Dunn index"].append(dunn_index)
      # data_frame["Davies Bouldin"].append(davies_bouldin_index)
      # data_frame["Silhouette index"].append(silhouette_index)
      data_frame["K"].append(neighbor)
      data_frame["TP"].append(TP)
      data_frame["TN"].append(TN)
      data_frame["FP"].append(FP)
      data_frame["FN"].append(FN)
      data_frame["pos_labeled_img"].append(pos_img)
      data_frame["neg_labeled_img"].append(neg_img)
      data_frame["corrected_count"].append(count)

      print(f"Labeled image: {size} \t Dataset: d_{select} \t K: {neighbor} \t Accuracy: {accuracy} \t Specificity: {specificity} \t Sensitivity: {sensitivity} \t AUC: {cl_auc} \t Corrected count: {count}")
      # select +=1 
      data_f=pd.DataFrame.from_dict(data_frame)
      data_f.to_csv(f"../knn_plot_csv/cs_vgg16_euclidean_dist_200_1210.csv",index=False)
    select +=1 

labeled data length 3
Unlabeled data length 3
Feature length neg: 20
Feature length pos: 20 


100%|██████████| 1072/1072 [00:38<00:00, 27.81it/s]


TP:  177
FP:  75
FN:  369
TN:  451
Labeled image: 240 	 Dataset: d_0 	 K: 5 	 Accuracy: 0.5858208900576409 	 Specificity: 0.857414432368547 	 Sensitivity: 0.32417581823853814 	 AUC: 0.6491124528196772 	 Corrected count: 74
Feature length neg: 20
Feature length pos: 20 


100%|██████████| 1072/1072 [00:38<00:00, 27.70it/s]


TP:  118
FP:  31
FN:  428
TN:  495
Labeled image: 240 	 Dataset: d_0 	 K: 9 	 Accuracy: 0.5718283528747355 	 Specificity: 0.9410646208923076 	 Sensitivity: 0.21611721215902543 	 AUC: 0.6479390381481636 	 Corrected count: 68
Feature length neg: 20
Feature length pos: 20 


...

TP:  20
FP:  8
FN:  15
TN:  19
Labeled image: 1250 	 Dataset: d_0 	 K: 25 	 Accuracy: 0.6290321566077166 	 Specificity: 0.7037034430727989 	 Sensitivity: 0.5714284081633119 	 AUC: 0.671957671957672 	 Corrected count: 419
Feature length neg: 20
Feature length pos: 20 


100%|██████████| 62/62 [00:03<00:00, 18.94it/s]


TP:  20
FP:  5
FN:  15
TN:  22
Labeled image: 1250 	 Dataset: d_0 	 K: 31 	 Accuracy: 0.677419245577541 	 Specificity: 0.8148145130316619 	 Sensitivity: 0.5714284081633119 	 AUC: 0.6671957671957671 	 Corrected count: 429
Feature length neg: 20
Feature length pos: 20 


100%|██████████| 62/62 [00:03<00:00, 18.40it/s]

TP:  20
FP:  5
FN:  15
TN:  22
Labeled image: 1250 	 Dataset: d_0 	 K: 39 	 Accuracy: 0.677419245577541 	 Specificity: 0.8148145130316619 	 Sensitivity: 0.5714284081633119 	 AUC: 0.6518518518518519 	 Corrected count: 415


In [22]:
s_model= "VGG16"
s_distance="Euclidean_dist"
len(labeled_data[select])

200

In [ ]:
data_f=pd.DataFrame.from_dict(data_frame)
data_f.to_csv(f"./csv_results_x-ray_counts/{s_model}_{s_distance}_dist_1100_1300.csv",index=False)